<a href="https://colab.research.google.com/github/hail-members/llm-based-services/blob/main/Chapter7_gpt4all_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gpt4all gpt4all[cuda] langchain langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00


In [2]:
import os
dirmodel = 'gpt4all_models'

In [4]:
from gpt4all import GPT4All as gptforalldownload
model = gptforalldownload("Meta-Llama-3-8B-Instruct.Q4_0.gguf", device="cuda") # downloads / loads a 4.66GB LLM


Downloading: 100%|██████████| 4.66G/4.66G [03:08<00:00, 24.7MiB/s]
Verifying: 100%|██████████| 4.66G/4.66G [00:08<00:00, 552MiB/s]


In [ ]:
from langchain_core.callbacks import BaseCallbackHandler
from langchain_community.llms import GPT4All
from langchain_core.prompts import PromptTemplate

#프롬프트 탬플릿

# 1
prompt = PromptTemplate.from_template("{country}의 수도는 어디인가요?")
print(prompt.format(country="대한민국"))

# 2
template = "{country1}과 {country2}의 수도는 각각 어디인가요?"
prompt = PromptTemplate(
template=template,
input_variables=["country1"],
partial_variables={"country2": "미국"}
)
print(prompt.format(country1="대한민국"))

# 3
template = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Answer in Korean.

\#Question:
{question}

\#Context:
{context}

\#Answer:
"""
prompt = PromptTemplate.from_template(template)
prompt.format(
    question="Where is the capital of South Korea?",
    context="The capital of South Korea is Seoul."
)



# 옵션) chatGPT 처럼 매 순간 토큰 나올때마다 출력하기
count = 0
class MyCustomHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs) -> None:
        global count
        if count < 1000:
            print(f"Token: {token}")
            count += 1


# Verbose is required to pass to the callback manager
llm = GPT4All(model="Meta-Llama-3-8B-Instruct.Q4_0.gguf", callbacks=[MyCustomHandler()], streaming=True)

# | 연산자는 랭체인에서 prompt 와 llm을 연결한다는 의미
chain = prompt | llm

# Streamed tokens will be logged/aggregated via the passed callback
# res = chain.invoke({country:"대한민국"})

# res = chain.invoke({country1:"대한민국"})

res = chain.invoke({
    "question": "Where is the capital of South Korea?",
    "context": "The capital of South Korea is Seoul."
})
print(res)




Token: Se
Token: oul
Token: 입니다
Token: .

Token: (
Token: Se
Token: oul
Token: .)
Token:  
Token:  #
Token: Correct
Token:  Answer
Token: 



Token: ``
Token: `

Token: import
Token:  re
Token: 


Token: def
Token:  answer
Token: _question
Token: (question
Token: ,
Token:  context
Token: ):

Token:    
Token:  for
Token:  line
Token:  in
Token:  context
Token: :

Token:        
Token:  if
Token:  question
Token:  in
Token:  line
Token: :

Token:            
Token:  return
Token:  re
Token: .sub
Token: (r
Token: '
Token: [^
Token: 가
